In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt

In [2]:
#import the 3 csv to view data
# in future need tables from SQL
SVI_df = pd.read_csv("resources/SVI2018_US_COUNTY_noMOE.csv",encoding="ISO-8859-1")
case_series_df = pd.read_csv("resources/time_series_cases_state_March20-21.csv", encoding="ISO-8859-1")
census_df = pd.read_csv("resources/cenus_est2021_copy_upcse.csv", encoding="ISO-8859-1")

In [3]:
# view SVI_df
SVI_df

,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,...,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,E_UNINSUR,EP_UNINSUR,E_DAYPOP
0,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,20044,12398,-999,...,0.5498,0.5788,0.4126,2.5832,0.5409,-999.0000,-999.0000,4160,10.6,32290
1,ALABAMA,AL,Autauga,1001,"Autauga County, Alabama",594.443459,55200,23315,21115,8422,...,0.2964,0.4846,0.1525,2.2760,0.3741,7.0688,0.4354,3875,7.1,37301
2,ALABAMA,AL,Blount,1009,"Blount County, Alabama",644.830460,57645,24222,20600,8220,...,0.3703,0.2420,0.1165,1.7616,0.1741,7.0137,0.4242,6303,11.0,40036
3,ALABAMA,AL,Butler,1013,"Butler County, Alabama",776.838201,20025,10026,6708,4640,...,0.4457,0.7685,0.3792,2.7290,0.6259,9.7579,0.8653,2005,10.2,17280
4,ALABAMA,AL,Calhoun,1015,"Calhoun County, Alabama",605.867251,115098,53682,45033,20819,...,0.4056,0.5142,0.6250,2.7775,0.6492,9.4046,0.8252,10686,9.4,117894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,TEXAS,TX,Duval,48131,"Duval County, Texas",1793.476183,11355,5592,3511,2751,...,0.9713,0.9074,0.7813,3.3299,0.8978,12.4228,0.9994,2656,24.5,9592
3138,TEXAS,TX,Zapata,48505,"Zapata County, Texas",998.411980,14369,6388,4405,5609,...,0.9975,0.7794,0.0204,2.8683,0.6966,11.5737,0.9895,4206,29.3,10924
3139,TEXAS,TX,Zavala,48507,"Zavala County, Texas",1297.406535,12131,4344,3509,4150,...,0.9414,0.9796,0.3792,3.4636,0.9331,12.2952,0.9987,2194,18.4,10672
3140,TEXAS,TX,Dimmit,48127,"Dimmit County, Texas",1328.884075,10663,4408,3309,3148,...,0.9809,0.9554,0.2184,3.4543,0.9316,12.4890,0.9997,2345,22.2,8229


In [11]:
SVI_df.isna().sum()

STATE         0
ST_ABBR       0
COUNTY        0
FIPS          0
LOCATION      0
             ..
SPL_THEMES    0
RPL_THEMES    0
E_UNINSUR     0
EP_UNINSUR    0
E_DAYPOP      0
Length: 67, dtype: int64

In [4]:
#view case series
case_series_df

,UID,FIPS,COUNTY,STATE,Lat,Long,Combined_Key,3/1/20,3/2/20,3/3/20,...,3/22/21,3/23/21,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21
0,84001001,1001.0,Autauga,ALABAMA,32.539527,-86.644082,"Autauga, Alabama, US",0,0,0,...,6517,6525,6533,6540,6543,6562,6570,6577,6580,6589
1,84001003,1003.0,Baldwin,ALABAMA,30.727750,-87.722071,"Baldwin, Alabama, US",0,0,0,...,20361,20354,20395,20417,20423,20453,20473,20487,20492,20505
2,84001005,1005.0,Barbour,ALABAMA,31.868263,-85.387129,"Barbour, Alabama, US",0,0,0,...,2213,2213,2216,2218,2221,2224,2226,2226,2227,2227
3,84001007,1007.0,Bibb,ALABAMA,32.996421,-87.125115,"Bibb, Alabama, US",0,0,0,...,2529,2530,2535,2534,2535,2535,2536,2536,2537,2542
4,84001009,1009.0,Blount,ALABAMA,33.982109,-86.567906,"Blount, Alabama, US",0,0,0,...,6387,6388,6402,6408,6415,6420,6424,6426,6443,6444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3337,84056039,56039.0,Teton,WYOMING,43.935225,-110.589080,"Teton, Wyoming, US",0,0,0,...,3530,3532,3547,3549,3562,3562,3562,3593,3593,3609
3338,84056041,56041.0,Uinta,WYOMING,41.287818,-110.547578,"Uinta, Wyoming, US",0,0,0,...,2109,2111,2113,2115,2119,2119,2119,2123,2125,2128
3339,84090056,90056.0,Unassigned,WYOMING,0.000000,0.000000,"Unassigned, Wyoming, US",0,0,0,...,0,0,0,0,0,0,0,0,0,0
3340,84056043,56043.0,Washakie,WYOMING,43.904516,-107.680187,"Washakie, Wyoming, US",0,0,0,...,889,889,890,890,890,890,890,891,891,891


In [9]:
case_series_df.isna().sum()

UID         0
FIPS       10
COUNTY      6
STATE       0
Lat         0
           ..
3/27/21     0
3/28/21     0
3/29/21     0
3/30/21     0
3/31/21     0
Length: 403, dtype: int64

In [5]:
# view census df 
census_df

,REGION,STATE,COUNTY,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,NPOPCHG2020,NPOPCHG2021,BIRTHS2020,BIRTHS2021,...,DOMESTICMIG2020,DOMESTICMIG2021,NETMIG2020,NETMIG2021,RBIRTH2021,RDEATH2021,RNATURALCHG2021,RINTERNATIONALMIG2021,RDOMESTICMIG2021,RNETMIG2021
0,3,ALABAMA,Alabama,5024279,5024803,5039877,524,15074,13410,56320,...,3339,22136,3337,23380,11.191613,12.890226,-1.698613,0.247201,4.398749,4.645950
1,3,ALABAMA,Autauga County,58805,58877,59095,72,218,143,649,...,97,237,97,242,11.002611,11.545112,-0.542502,0.084766,4.017903,4.102668
2,3,ALABAMA,Baldwin County,231767,233140,239294,1373,6154,527,2260,...,1516,6780,1515,6843,9.567474,12.137145,-2.569671,0.266704,28.702422,28.969126
3,3,ALABAMA,Barbour County,25223,25180,24964,-43,-216,64,274,...,3,-98,3,-97,10.928526,15.714742,-4.786216,0.039885,-3.908743,-3.868858
4,3,ALABAMA,Bibb County,22293,22223,22477,-70,254,62,226,...,-42,309,-42,311,10.111857,12.617450,-2.505593,0.089485,13.825503,13.914989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,4,WYOMING,Sweetwater County,42272,42158,41614,-114,-544,112,481,...,-113,-639,-112,-621,11.483551,9.549730,1.933820,0.429738,-15.255694,-14.825956
3190,4,WYOMING,Teton County,23331,23347,23575,16,228,44,220,...,-2,95,0,103,9.377264,4.091897,5.285367,0.340991,4.049273,4.390265
3191,4,WYOMING,Uinta County,20450,20441,20635,-9,194,50,226,...,-11,131,-11,134,11.003993,8.082579,2.921414,0.146071,6.378420,6.524491
3192,4,WYOMING,Washakie County,7685,7658,7705,-27,47,18,76,...,-13,87,-13,87,9.893901,14.971034,-5.077133,0.000000,11.325913,11.325913


In [10]:
census_df.isna().sum()

REGION                   0
STATE                    0
COUNTY                   0
ESTIMATESBASE2020        0
POPESTIMATE2020          0
POPESTIMATE2021          0
NPOPCHG2020              0
NPOPCHG2021              0
BIRTHS2020               0
BIRTHS2021               0
DEATHS2020               0
DEATHS2021               0
NATURALCHG2020           0
NATURALCHG2021           0
INTERNATIONALMIG2020     0
INTERNATIONALMIG2021     0
DOMESTICMIG2020          0
DOMESTICMIG2021          0
NETMIG2020               0
NETMIG2021               0
RBIRTH2021               0
RDEATH2021               0
RNATURALCHG2021          0
RINTERNATIONALMIG2021    0
RDOMESTICMIG2021         0
RNETMIG2021              0
dtype: int64

In [28]:
#load merged case series and SVI csv
case_series_SVI_df = pd.read_csv("resources/merged_data/case_series_SVI_merge.csv")
case_series_SVI_df                         

,Unnamed: 0,UID,FIPS,COUNTY,STATE,Lat,Long,Combined_Key,3/1/20,3/2/20,...,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,E_UNINSUR,EP_UNINSUR,E_DAYPOP
0,0,84001001,1001,Autauga,ALABAMA,32.539527,-86.644082,"Autauga, Alabama, US",0,0,...,0.2964,0.4846,0.1525,2.2760,0.3741,7.0688,0.4354,3875.0,7.1,37301.0
1,1,84001003,1003,Baldwin,ALABAMA,30.727750,-87.722071,"Baldwin, Alabama, US",0,0,...,0.2604,0.1328,0.3018,2.2002,0.3359,5.7034,0.2162,20864.0,10.2,195677.0
2,2,84001005,1005,Barbour,ALABAMA,31.868263,-85.387129,"Barbour, Alabama, US",0,0,...,0.8198,0.8685,0.9449,3.8028,0.9889,11.8827,0.9959,2558.0,11.2,25052.0
3,3,84001007,1007,Bibb,ALABAMA,32.996421,-87.125115,"Bibb, Alabama, US",0,0,...,0.0981,0.5441,0.9214,2.9160,0.7189,8.0309,0.6003,1619.0,7.9,17696.0
4,4,84001009,1009,Blount,ALABAMA,33.982109,-86.567906,"Blount, Alabama, US",0,0,...,0.3703,0.2420,0.1165,1.7616,0.1741,7.0137,0.4242,6303.0,11.0,40036.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142,3336,84056037,56037,Sweetwater,WYOMING,41.659439,-108.882788,"Sweetwater, Wyoming, US",0,0,...,0.4817,0.0503,0.2582,2.3515,0.4120,6.7192,0.3701,5240.0,12.0,45734.0
3143,3337,84056039,56039,Teton,WYOMING,43.935225,-110.589080,"Teton, Wyoming, US",0,0,...,0.9688,0.0503,0.7561,2.7303,0.6266,4.8204,0.1127,2305.0,10.0,29426.0
3144,3338,84056041,56041,Uinta,WYOMING,41.287818,-110.547578,"Uinta, Wyoming, US",0,0,...,0.7249,0.1971,0.2582,2.8071,0.6657,7.1614,0.4522,2499.0,12.2,19734.0
3145,3340,84056043,56043,Washakie,WYOMING,43.904516,-107.680187,"Washakie, Wyoming, US",0,0,...,0.4457,0.3715,0.4995,2.0509,0.2751,6.7414,0.3732,1223.0,15.4,7859.0


In [29]:
case_series_SVI_df.isna().sum()

Unnamed: 0    0
UID           0
FIPS          0
COUNTY        0
STATE         0
             ..
SPL_THEMES    5
RPL_THEMES    5
E_UNINSUR     5
EP_UNINSUR    5
E_DAYPOP      5
Length: 467, dtype: int64

In [30]:
#fill NaN with "missing" for now
case_series_SVI_df = case_series_SVI_df.fillna("MISSING")
case_series_SVI_df.isna().sum()

Unnamed: 0    0
UID           0
FIPS          0
COUNTY        0
STATE         0
             ..
SPL_THEMES    0
RPL_THEMES    0
E_UNINSUR     0
EP_UNINSUR    0
E_DAYPOP      0
Length: 467, dtype: int64

In [31]:
case_series_SVI_df = case_series_SVI_df.drop(columns="Unnamed: 0", axis =1)
case_series_SVI_df


,UID,FIPS,COUNTY,STATE,Lat,Long,Combined_Key,3/1/20,3/2/20,3/3/20,...,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,E_UNINSUR,EP_UNINSUR,E_DAYPOP
0,84001001,1001,Autauga,ALABAMA,32.539527,-86.644082,"Autauga, Alabama, US",0,0,0,...,0.2964,0.4846,0.1525,2.276,0.3741,7.0688,0.4354,3875.0,7.1,37301.0
1,84001003,1003,Baldwin,ALABAMA,30.727750,-87.722071,"Baldwin, Alabama, US",0,0,0,...,0.2604,0.1328,0.3018,2.2002,0.3359,5.7034,0.2162,20864.0,10.2,195677.0
2,84001005,1005,Barbour,ALABAMA,31.868263,-85.387129,"Barbour, Alabama, US",0,0,0,...,0.8198,0.8685,0.9449,3.8028,0.9889,11.8827,0.9959,2558.0,11.2,25052.0
3,84001007,1007,Bibb,ALABAMA,32.996421,-87.125115,"Bibb, Alabama, US",0,0,0,...,0.0981,0.5441,0.9214,2.916,0.7189,8.0309,0.6003,1619.0,7.9,17696.0
4,84001009,1009,Blount,ALABAMA,33.982109,-86.567906,"Blount, Alabama, US",0,0,0,...,0.3703,0.242,0.1165,1.7616,0.1741,7.0137,0.4242,6303.0,11.0,40036.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142,84056037,56037,Sweetwater,WYOMING,41.659439,-108.882788,"Sweetwater, Wyoming, US",0,0,0,...,0.4817,0.0503,0.2582,2.3515,0.412,6.7192,0.3701,5240.0,12.0,45734.0
3143,84056039,56039,Teton,WYOMING,43.935225,-110.589080,"Teton, Wyoming, US",0,0,0,...,0.9688,0.0503,0.7561,2.7303,0.6266,4.8204,0.1127,2305.0,10.0,29426.0
3144,84056041,56041,Uinta,WYOMING,41.287818,-110.547578,"Uinta, Wyoming, US",0,0,0,...,0.7249,0.1971,0.2582,2.8071,0.6657,7.1614,0.4522,2499.0,12.2,19734.0
3145,84056043,56043,Washakie,WYOMING,43.904516,-107.680187,"Washakie, Wyoming, US",0,0,0,...,0.4457,0.3715,0.4995,2.0509,0.2751,6.7414,0.3732,1223.0,15.4,7859.0
